In [1]:
# Updating sklearn to prevent version mismatches
# !pip install sklearn --upgrade

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Reading CSV
df = pd.read_csv("source_data/exoplanet_data.csv")
# Dropping the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Dropping the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
# Setting features/x values
selected_features = df.drop("koi_disposition", axis=1)

In [5]:
# Setting y value
y = df["koi_disposition"]
print(selected_features.shape, y.shape)

(6991, 40) (6991,)


In [6]:
# Creating test/train split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=21)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6966,1,0,1,0,361.901618,0.008898,-0.008898,405.302100,0.020000,-0.020000,...,-136,4.603,0.040,-0.060,0.717,0.086,-0.058,290.37891,39.694000,12.673
1714,0,0,0,0,6.739683,0.000064,-0.000064,132.292960,0.008160,-0.008160,...,-71,4.261,0.168,-0.112,1.176,0.183,-0.203,280.82339,42.636051,14.455
225,0,0,0,0,3.166354,0.000003,-0.000003,170.966145,0.000631,-0.000631,...,-211,4.470,0.052,-0.208,0.998,0.318,-0.106,286.36011,45.414291,15.603
5266,0,0,0,0,25.090157,0.000482,-0.000482,138.498800,0.016200,-0.016200,...,-183,3.985,0.266,-0.114,1.788,0.351,-0.571,298.67377,43.795479,12.014
5468,0,1,0,0,7.234966,0.000002,-0.000002,134.582307,0.000207,-0.000207,...,-186,4.573,0.040,-0.160,0.798,0.206,-0.069,290.22617,37.759640,16.981


In [7]:
# Scaling the data
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Label encoding
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [9]:
# One-hot encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
# Training the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=40))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                410       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 33        
Total params: 553
Trainable params: 553
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Fitting the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
164/164 - 1s - loss: 0.7534 - accuracy: 0.6914
Epoch 2/100
164/164 - 0s - loss: 0.5062 - accuracy: 0.7692
Epoch 3/100
164/164 - 0s - loss: 0.4060 - accuracy: 0.8177
Epoch 4/100
164/164 - 0s - loss: 0.3464 - accuracy: 0.8478
Epoch 5/100
164/164 - 0s - loss: 0.3098 - accuracy: 0.8648
Epoch 6/100
164/164 - 0s - loss: 0.2870 - accuracy: 0.8787
Epoch 7/100
164/164 - 0s - loss: 0.2745 - accuracy: 0.8867
Epoch 8/100
164/164 - 0s - loss: 0.2661 - accuracy: 0.8913
Epoch 9/100
164/164 - 0s - loss: 0.2609 - accuracy: 0.8917
Epoch 10/100
164/164 - 0s - loss: 0.2556 - accuracy: 0.8947
Epoch 11/100
164/164 - 0s - loss: 0.2526 - accuracy: 0.8947
Epoch 12/100
164/164 - 0s - loss: 0.2495 - accuracy: 0.8961
Epoch 13/100
164/164 - 0s - loss: 0.2459 - accuracy: 0.8974
Epoch 14/100
164/164 - 0s - loss: 0.2434 - accuracy: 0.8997
Epoch 15/100
164/164 - 0s - loss: 0.2410 - accuracy: 0.8983
Epoch 16/100
164/164 - 0s - loss: 0.2372 - accuracy: 0.8999
Epoch 17/100
164/164 - 0s - loss: 0.2360 - accura

In [14]:
# Evaluating the model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.3182 - accuracy: 0.8781
Deep Neural Network - Loss: 0.3182031512260437, Accuracy: 0.8781464695930481


In [15]:
# Making inferences
encoded_predictions = np.argmax(model.predict(X_test_scaled[:10]), axis=-1)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [16]:
inference_df = pd.DataFrame({'Predicted': prediction_labels, 'Actual': (y_test[:10])}).sort_index()
inference_df

,Predicted,Actual
1039,CONFIRMED,CONFIRMED
2046,CANDIDATE,CONFIRMED
2693,CANDIDATE,CANDIDATE
3481,FALSE POSITIVE,FALSE POSITIVE
4104,FALSE POSITIVE,FALSE POSITIVE
4252,FALSE POSITIVE,FALSE POSITIVE
4496,FALSE POSITIVE,FALSE POSITIVE
4945,CANDIDATE,CANDIDATE
5819,FALSE POSITIVE,FALSE POSITIVE
6378,FALSE POSITIVE,FALSE POSITIVE


In [17]:
# Saving the model
model.save("deep_learning_model.h5")